<a href="https://colab.research.google.com/github/Natotela/Bemba/blob/master/Shap_e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/openai/shap-e.git

Cloning into 'shap-e'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 322 (delta 29), reused 13 (delta 13), pack-reused 278
Receiving objects: 100% (322/322), 11.71 MiB | 16.70 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [2]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61405 sha256=996bd006461a2af638194f7361a4d10c02fef4410571d11bf7379f81959666b7
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31531 sha256=7d6d37cf4eb6c3d6c85dec99325733738b7c5d6fd845812c0f9bba3c5dd00901
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt200/dow

In [3]:
%ls

sample_data/  shap-e/


In [4]:
%cd shap-e/


/content/shap-e


In [5]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-ijs8yguw/clip_59d2b5915a0e485b9fc408e7b8a54c56
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-ijs8yguw/clip_59d2b5915a0e485b9fc408e7b8a54c56
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl 

In [ ]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:06<00:00, 145MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [ ]:
batch_size = 10
guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
image = load_image("/content/5.png")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
render_mode = 'stf' # you can change this to 'stf' for mesh rendering
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2YzMzNmM2ZmMwAzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2YzMzNmM2ZmMwAzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMwBmMzNmM2ZmMwAzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2YzM2ZmMzMzZmYzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2ZmMwAzZjMzZmYzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMzNmM2ZmMwAzZjMzZm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMzNmM2ZmMzMzZmYzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2ZmMzMzZmYzZjNmZm…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzMzNmM2ZmMzMzZmYzZj…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAAAzADMzAAAAMzMAMwAzMzMzM2YzMzNmM2ZmMzMzZm…

In [ ]:
cameras

In [ ]:
gif_widget(images)

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAADMAAGYzAJkzAGZmAJlmAMxmAMyZADMzM5lmM8xmM8yZM2…

In [ ]:
#/content/shap-e/shap_e/examples/example_data/corgi.png

In [ ]:
%cd /content/

/content


In [ ]:
import os
os.mkdir("/content/res")

In [ ]:
%cd res

/content/res


In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)

In [ ]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html